In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests, time, os, pandas as pd

def crawl_event_page(url):
    save_dir = "알라딘"
    os.makedirs(save_dir, exist_ok=True)

    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)  # 로딩 대기

    # 스크롤 내려서 이미지 로딩 유도
    for _ in range(10):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    blocks = soup.select('table#Table1')
    data, count = [], 1

    print(f"🔍 이벤트 블록 수: {len(blocks)}")

    for block in blocks:
        img_tag = block.select_one('div.bn img')
        name_tag = block.select_one('a.ml h3')
        period_tag = block.select_one('span.date')

        if not (img_tag and name_tag and period_tag):
            continue

        img_url = img_tag.get('src')
        if img_url.startswith('//'):
            img_url = 'https:' + img_url
        elif img_url.startswith('/'):
            img_url = 'https://www.aladin.co.kr' + img_url

        event_name = name_tag.get_text(strip=True)
        period = period_tag.get_text(strip=True)

        img_name = f'image_{count:03d}.jpg'
        try:
            res = requests.get(img_url)
            if res.status_code == 200:
                with open(os.path.join(save_dir, img_name), 'wb') as f:
                    f.write(res.content)
                print(f'📸 저장됨: {img_name}')
        except Exception as e:
            print(f'❌ 이미지 저장 실패: {e}')

        data.append({
            '이미지명': img_name,
            '이벤트명': event_name,
            '증정내용': '',
            '기간': period
        })
        count += 1

    driver.quit()

    if data:
        df = pd.DataFrame(data)
        df.to_csv(os.path.join(save_dir, '알라딘이벤트정보.csv'), index=False, encoding='utf-8-sig')
        print(f"\n✅ 총 {len(df)}건 저장 완료 → 알라딘이벤트정보.csv")
    else:
        print("❌ 저장할 이벤트 없음")

# 실행
if __name__ == "__main__":
    crawl_event_page("https://www.aladin.co.kr/events/wevent_sub.aspx")


🔍 이벤트 블록 수: 2752
📸 저장됨: image_001.jpg
📸 저장됨: image_002.jpg
📸 저장됨: image_003.jpg
📸 저장됨: image_005.jpg
📸 저장됨: image_006.jpg
📸 저장됨: image_007.jpg
📸 저장됨: image_008.jpg
📸 저장됨: image_009.jpg
📸 저장됨: image_010.jpg
📸 저장됨: image_011.jpg
📸 저장됨: image_012.jpg
📸 저장됨: image_013.jpg
📸 저장됨: image_014.jpg
📸 저장됨: image_015.jpg
📸 저장됨: image_016.jpg
📸 저장됨: image_017.jpg
📸 저장됨: image_018.jpg
📸 저장됨: image_019.jpg
📸 저장됨: image_020.jpg
📸 저장됨: image_021.jpg
📸 저장됨: image_022.jpg
📸 저장됨: image_023.jpg
📸 저장됨: image_024.jpg
📸 저장됨: image_025.jpg
📸 저장됨: image_026.jpg
📸 저장됨: image_027.jpg
📸 저장됨: image_028.jpg
📸 저장됨: image_029.jpg
📸 저장됨: image_030.jpg
📸 저장됨: image_031.jpg
📸 저장됨: image_032.jpg
📸 저장됨: image_033.jpg
📸 저장됨: image_034.jpg
📸 저장됨: image_035.jpg
📸 저장됨: image_036.jpg
📸 저장됨: image_037.jpg
📸 저장됨: image_038.jpg
📸 저장됨: image_039.jpg
📸 저장됨: image_040.jpg
📸 저장됨: image_041.jpg
📸 저장됨: image_042.jpg
📸 저장됨: image_043.jpg
📸 저장됨: image_044.jpg
📸 저장됨: image_045.jpg
📸 저장됨: image_046.jpg
📸 저장됨: image_047.jpg
📸 저장됨: image_048.

In [66]:
import pandas as pd
al_event = pd.read_csv('알라딘이벤트정보.csv')

In [67]:
al_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   이미지명    2500 non-null   object 
 1   이벤트명    2500 non-null   object 
 2   증정내용    0 non-null      float64
 3   기간      2500 non-null   object 
dtypes: float64(1), object(3)
memory usage: 78.2+ KB


In [68]:
al_event['기간'] = al_event['기간'].str.replace('기간 :','')
al_event

,이미지명,이벤트명,증정내용,기간
0,image_001.jpg,"<길고양이와 늑대 5> 출간, PET 스탠드 2종 세트",NaN,5월 21일 ~
1,image_002.jpg,"A4 봉투형 파일(2종 중 택1, 푸른숲주니어 어린이 브랜드전 도서 1권 이상 구매 시)",NaN,5월 21일 ~
2,image_005.jpg,<여우의 전화박스> 여우 패브릭 책갈피,NaN,5월 21일 ~
3,image_006.jpg,방구석 노트 (이벤트 대상도서 구매 시),NaN,5월 21일 ~
4,image_007.jpg,내 인생을 바꾼 설교 수첩 (이벤트 대상도서 1만원 이상 구매 시),NaN,5월 21일 ~
...,...,...,...,...
2495,image_2748.jpg,PVC 파우치,NaN,10월 14일 ~
2496,image_2749.jpg,"작가특보 시리즈 3종 동시구매 시, 어느 날 글쓰기를 물어보고 싶을 때 책자",NaN,10월 8일 ~ 7월 30일
2497,image_2750.jpg,[편의점택배]GS배송 이용하면 적립금 300원,NaN,8월 3일 ~ 6월 30일
2498,image_2751.jpg,이달의 적립금 혜택,NaN,7월 1일 ~ 12월 31일


In [69]:
al_event2 = al_event.copy()

In [76]:
al_event2

,이미지명,이벤트명,증정내용,기간
0,image_001.jpg,"<길고양이와 늑대 5> 출간, PET 스탠드 2종 세트",NaN,5월 21일 ~
1,image_002.jpg,"A4 봉투형 파일(2종 중 택1, 푸른숲주니어 어린이 브랜드전 도서 1권 이상 구매 시)",NaN,5월 21일 ~
2,image_005.jpg,<여우의 전화박스> 여우 패브릭 책갈피,NaN,5월 21일 ~
3,image_006.jpg,방구석 노트 (이벤트 대상도서 구매 시),NaN,5월 21일 ~
4,image_007.jpg,내 인생을 바꾼 설교 수첩 (이벤트 대상도서 1만원 이상 구매 시),NaN,5월 21일 ~
...,...,...,...,...
2495,image_2748.jpg,PVC 파우치,NaN,10월 14일 ~
2496,image_2749.jpg,"작가특보 시리즈 3종 동시구매 시, 어느 날 글쓰기를 물어보고 싶을 때 책자",NaN,10월 8일 ~ 7월 30일
2497,image_2750.jpg,[편의점택배]GS배송 이용하면 적립금 300원,NaN,8월 3일 ~ 6월 30일
2498,image_2751.jpg,이달의 적립금 혜택,NaN,7월 1일 ~ 12월 31일


In [77]:
bf_index = al_event2[al_event2['이벤트명'].str.contains('알라딘×KBS 라디오 최고의 클립', regex=False, na=False)].index.tolist()

In [78]:
al_event2 = al_event2.drop(bf_index).reset_index(drop=True)

In [80]:
al_event2[al_event2['이벤트명'].str.contains('[편의점택배]', regex=False, na=False)].index.tolist()

[2497]

In [82]:
al_event2 = al_event2.drop(2497)

In [ ]:
al_event2[al_event2['이벤트명'].str.contains('이달의', regex=False, na=False)].index.tolist()

[245, 255, 261, 263, 534, 2498]

In [84]:
al_event2 = al_event2.drop([245, 255, 261, 263, 534, 2498])

In [88]:
al_event2[al_event2['이벤트명'].str.contains('인디고', regex=False, na=False)].index.tolist()

[57, 102, 2081]

In [90]:
al_event2 = al_event2.drop([57, 102, 2081])

In [92]:
al_event2[al_event2['이벤트명'].str.contains('대선', regex=False, na=False)].index.tolist()

[507]

In [93]:
al_event2 = al_event2.drop(507)

In [94]:
al_event2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2489 entries, 0 to 2499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   이미지명    2489 non-null   object 
 1   이벤트명    2489 non-null   object 
 2   증정내용    0 non-null      float64
 3   기간      2489 non-null   object 
dtypes: float64(1), object(3)
memory usage: 97.2+ KB


In [95]:
al_event2.to_csv('알라딘이벤트정보.csv', index=False)